In [1]:
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
from qiskit.visualization import *
from qiskit_ibm_runtime import *
import numpy as np
from qiskit import *
from qiskit_aer import *
from scipy.fft import fft
from scipy.stats import entropy
from qiskit_aer.noise import *

# Function to generate noisy QRNG data
def generate_qrng_data(num_bits, noise_model):
    circuit = QuantumCircuit(1, 1)
    circuit.h(0)  # Apply Hadamard gate
    circuit.measure(0, 0)  # Measure the qubit

    simulator = Aer.get_backend('aer_simulator')
    circuit = transpile(circuit, simulator)
    result = simulator.run(circuit, shots=num_bits, noise_model=noise_model).result()
    counts = result.get_counts()

    # Convert counts to binary array
    qrng_data = []
    for bitstring, count in counts.items():
        qrng_data.extend([int(bitstring)] * count)
    return np.array(qrng_data)

# Function to create a Toeplitz matrix
def create_toeplitz(first_row, first_column):
    size = len(first_row)
    toeplitz_matrix = np.zeros((size, size), dtype=int)
    for i in range(size):
        toeplitz_matrix[i, :size - i] = first_row[:size - i]
        if i > 0:
            toeplitz_matrix[i, size - i:] = first_column[1:i + 1]
    return toeplitz_matrix

# Min-entropy calculation
def min_entropy(data):
    counts = np.bincount(data)
    probabilities = counts / len(data)
    probabilities = probabilities[probabilities > 0]
    return -np.log2(np.max(probabilities))

# Collision entropy calculation
def collision_entropy(data):
    counts = np.bincount(data)
    probabilities = counts / len(data)
    return -np.log2(np.sum(probabilities**2))

# Define noise model
noise_model = NoiseModel()
depolarizing_err = depolarizing_error(0.01, 1)  # 1% depolarizing error
noise_model.add_all_qubit_quantum_error(depolarizing_err, ['h'])

# Main script
num_bits = 1024

# Step 1: Generate QRNG data with noise
qrng_data = generate_qrng_data(num_bits, noise_model)

# Step 2: Define Toeplitz matrix
toeplitz_size = len(qrng_data)
first_row = np.random.randint(0, 2, toeplitz_size)
first_column = np.random.randint(0, 2, toeplitz_size)
toeplitz_matrix = create_toeplitz(first_row, first_column)

# Step 3: Apply Toeplitz transformation
transformed_data = (toeplitz_matrix @ qrng_data) % 2

# Step 4: Apply FFT-based Toeplitz transformation
fft_transformed_data = np.real(fft((toeplitz_matrix @ qrng_data) % 2)) > 0.5
fft_transformed_data = fft_transformed_data.astype(int)

# Step 5: Calculate entropy metrics
original_shannon_entropy = entropy(np.bincount(qrng_data) / len(qrng_data), base=2)
transformed_shannon_entropy = entropy(np.bincount(transformed_data) / len(transformed_data), base=2)
fft_transformed_shannon_entropy = entropy(np.bincount(fft_transformed_data) / len(fft_transformed_data), base=2)

original_min_entropy = min_entropy(qrng_data)
transformed_min_entropy = min_entropy(transformed_data)
fft_transformed_min_entropy = min_entropy(fft_transformed_data)

original_collision_entropy = collision_entropy(qrng_data)
transformed_collision_entropy = collision_entropy(transformed_data)
fft_transformed_collision_entropy = collision_entropy(fft_transformed_data)

# Output results
print("Shannon Entropy:")
print(f"Original: {original_shannon_entropy}")
print(f"Transformed: {transformed_shannon_entropy}")
print(f"FFT Transformed: {fft_transformed_shannon_entropy}")

print("\nMin-Entropy:")
print(f"Original: {original_min_entropy}")
print(f"Transformed: {transformed_min_entropy}")
print(f"FFT Transformed: {fft_transformed_min_entropy}")

print("\nCollision Entropy:")
print(f"Original: {original_collision_entropy}")
print(f"Transformed: {transformed_collision_entropy}")
print(f"FFT Transformed: {fft_transformed_collision_entropy}")


Shannon Entropy:
Original: 0.9999009357256292
Transformed: 0.783888662623139
FFT Transformed: 0.9962296063061024

Min-Entropy:
Original: 0.9831917123134462
Transformed: 0.38345115622101067
FFT Transformed: 0.8993376609948014

Collision Entropy:
Original: 0.9998018895890571
Transformed: 0.6390114988674844
FFT Transformed: 0.9924853888509874


In [3]:
from scipy.special import *

# Function to generate noisy QRNG data
def generate_qrng_data(num_bits, noise_model):
    circuit = QuantumCircuit(1, 1)
    circuit.h(0)  # Apply Hadamard gate
    circuit.measure(0, 0)  # Measure the qubit

    simulator = Aer.get_backend('aer_simulator')
    circuit = transpile(circuit, simulator)
    result = simulator.run(circuit, shots=num_bits, noise_model=noise_model).result()
    counts = result.get_counts()

    # Convert counts to binary array
    qrng_data = []
    for bitstring, count in counts.items():
        qrng_data.extend([int(bitstring)] * count)
    return np.array(qrng_data)

# Function to create a Toeplitz matrix
def create_toeplitz(first_row, first_column):
    size = len(first_row)
    toeplitz_matrix = np.zeros((size, size), dtype=int)
    for i in range(size):
        toeplitz_matrix[i, :size - i] = first_row[:size - i]
        if i > 0:
            toeplitz_matrix[i, size - i:] = first_column[1:i + 1]
    return toeplitz_matrix

# Min-entropy calculation
def min_entropy(data):
    counts = np.bincount(data)
    probabilities = counts / len(data)
    probabilities = probabilities[probabilities > 0]
    return -np.log2(np.max(probabilities))

# Collision entropy calculation
def collision_entropy(data):
    counts = np.bincount(data)
    probabilities = counts / len(data)
    return -np.log2(np.sum(probabilities**2))

# Define noise model
noise_model = NoiseModel()

# Depolarizing error
depolarizing_err = depolarizing_error(0.01, 1)  # 1% depolarizing error
noise_model.add_all_qubit_quantum_error(depolarizing_err, ['h'])

# Amplitude damping error
amplitude_damping_err = amplitude_damping_error(0.01)
noise_model.add_all_qubit_quantum_error(amplitude_damping_err, ['h'])

# Phase damping error
phase_damping_err = phase_damping_error(0.01)
noise_model.add_all_qubit_quantum_error(phase_damping_err, ['h'])

# Main script
num_bits = 1024

# Step 1: Generate QRNG data with noise
qrng_data = generate_qrng_data(num_bits, noise_model)

# Step 2: Define Toeplitz matrix
toeplitz_size = len(qrng_data)
first_row = np.random.randint(0, 2, toeplitz_size)
first_column = np.random.randint(0, 2, toeplitz_size)
toeplitz_matrix = create_toeplitz(first_row, first_column)

# Step 3: Apply Toeplitz transformation
transformed_data = (toeplitz_matrix @ qrng_data) % 2

# Step 4: Apply FFT-based Toeplitz transformation
fft_transformed_data = np.real(fft((toeplitz_matrix @ qrng_data) % 2)) > 0.5
fft_transformed_data = fft_transformed_data.astype(int)

# Step 5: Calculate entropy metrics
original_shannon_entropy = entropy(np.bincount(qrng_data) / len(qrng_data), base=2)
transformed_shannon_entropy = entropy(np.bincount(transformed_data) / len(transformed_data), base=2)
fft_transformed_shannon_entropy = entropy(np.bincount(fft_transformed_data) / len(fft_transformed_data), base=2)

original_min_entropy = min_entropy(qrng_data)
transformed_min_entropy = min_entropy(transformed_data)
fft_transformed_min_entropy = min_entropy(fft_transformed_data)

original_collision_entropy = collision_entropy(qrng_data)
transformed_collision_entropy = collision_entropy(transformed_data)
fft_transformed_collision_entropy = collision_entropy(fft_transformed_data)

# Output results
print("Shannon Entropy:")
print(f"Original: {original_shannon_entropy}")
print(f"Transformed: {transformed_shannon_entropy}")
print(f"FFT Transformed: {fft_transformed_shannon_entropy}")

print("\nMin-Entropy:")
print(f"Original: {original_min_entropy}")
print(f"Transformed: {transformed_min_entropy}")
print(f"FFT Transformed: {fft_transformed_min_entropy}")

print("\nCollision Entropy:")
print(f"Original: {original_collision_entropy}")
print(f"Transformed: {transformed_collision_entropy}")
print(f"FFT Transformed: {fft_transformed_collision_entropy}")



Shannon Entropy:
Original: 0.9916599563439634
Transformed: 0.7560809745687453
FFT Transformed: 0.9964308234404787

Min-Entropy:
Original: 0.8527950750577712
Transformed: 0.3543415675912899
FFT Transformed: 0.9019679170394733

Collision Entropy:
Original: 0.9834474007929441
Transformed: 0.6009849933621354
FFT Transformed: 0.9928851085172901
